In [1]:
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as mc
import warnings
import numpy as np
from tqdm import tqdm
import spacy

In [51]:
def Add_subset(k, scores, test_idx, pred, X, y, thresh):
    #print("test_idx", test_idx)
    #print("old")
    #print(pred[test_idx])
    
    if pred[test_idx] > thresh:
        top_k_index = scores[test_idx].argsort()[-k:]
    else:
        top_k_index = scores[test_idx].argsort()[:k]
        
    y_k = y["train_add"][top_k_index]
    X_k = X["train_add"][top_k_index]
    y_added = np.concatenate((y["train"],y_k))
    X_added = np.concatenate((X["train"],X_k), axis=0)

    prediction = -np.sum(scores[test_idx][top_k_index])
    #print("prediction", prediction)

    return X_added, y_added, prediction, top_k_index


def loss_gradient(X, y, model):
    F_train = np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)
    error_train = model.predict_proba(X)[:, 1] - y
    gradient_train = F_train * error_train[:, None]
    return gradient_train

In [52]:
def approximate_k(test_idx, pred, delta_pred, y, thresh):
    old = pred[test_idx].item()
    
    if pred[test_idx] > thresh:
        top_k_index = np.flip(delta_pred[test_idx].argsort())
    else:
        top_k_index = delta_pred[test_idx].argsort()
    
    for k in range(1, y["train"].shape[0]):
        change = -np.sum(delta_pred[test_idx][top_k_index[:k]])
        
        if old > thresh and old + change < thresh:
            return k
        elif old < thresh and old + change > thresh:
            return k
        
    return None

In [53]:
def new_train(k, dev_index, scores, l2, X, model, pred, y, thresh):
    X_k, y_k, prediction, top_k_index = Add_subset(k, scores, dev_index, pred, X, y, thresh)
    
    if y_k.shape[0] == np.sum(y_k) or np.sum(y_k) == 0: # data contains only one class: 1
        return None, None, None

    # Fit the model again
    model_k = LogisticRegression(penalty='l2', C=1/l2)
    model_k.fit(X_k, y_k)

    # predictthe probaility with test point
    test_point = X["dev"][dev_index]
    test_point=np.reshape(test_point, (1,-1))
    
    new = model_k.predict_proba(test_point)[0][1]
    change = -(model.predict_proba(test_point)[0][1] - new)
    #change = model_k.predict_proba(test_point)[0][1]-model.predict_proba(test_point)[0][1]
    flip = (model.predict(test_point) == model_k.predict(test_point))
    
    """
    print("change    ", change)
    print("old       ", model.predict_proba(test_point)[0][1])
    print()
    """
    error = np.abs((change - prediction)/prediction)
    return change, flip, prediction,new, error, top_k_index

In [48]:
def IP(X, y, l2, dataname, thresh, modi=None):
    model = LogisticRegression(penalty='l2', C=1/l2)
    model.fit(X["train"], y["train"])
    pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))

    gradient_train_add = loss_gradient(X["train_add"], y["train_add"], model)

    w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
    F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))],
                             axis=1)  # Concatenating one to calculate the gradient with respect to intercept
    F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

    # error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
    # error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

    # gradient_train = F_train * error_train[:, None]
    # gradient_dev = F_dev * error_dev[:, None]

    probs = model.predict_proba(X["train"])[:, 1]
    hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + l2 * np.eye(F_train.shape[1]) / \
              X["train"].shape[0]
    inverse_hessian = np.linalg.inv(hessian)
    eps = 1 / X["train"].shape[0]
    delta_k = -eps * inverse_hessian @ (- gradient_train_add).T
    grad_f = F_dev * (pred * (1 - pred))
    delta_pred = grad_f @ delta_k
    return delta_pred
    


    


In [33]:
import spacy
sst_dataset = {}
for split in ["train", "dev", "test"]:
    URL = f"https://raw.githubusercontent.com/successar/instance_attributions_NLP/master/Datasets/SST/data/{split}.jsonl"
    import urllib.request, json
    with urllib.request.urlopen(URL) as url:
        data = url.read().decode()
        data = [json.loads(line) for line in data.strip().split("\n")]
        sst_dataset[split] = data


nlp = spacy.load('en_core_web_sm')

X, y = {}, {}
for split in ["train", "dev"]:
    X[split] = np.array([nlp(example["document"]).vector for example in tqdm(sst_dataset[split])])
    y[split] = np.array([example["label"] for example in sst_dataset[split]])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 872/872 [00:05<00:00, 173.12it/s]


In [34]:
from sklearn.model_selection import train_test_split
X_train, X_train_add, y_train, y_train_add = train_test_split(
    X["train"], y["train"], test_size=0.4, random_state=42)

In [35]:
X["train"] = X_train
X["train_add"] = X_train_add
y["train"] = y_train
y["train_add"]= y_train_add
X["train"].shape, X["train_add"].shape, X["dev"].shape

((4152, 96), (2768, 96), (872, 96))

In [36]:
thresh = 0.5
l2 = 1000
model = LogisticRegression(penalty='l2', C=1/l2)
model.fit(X["train"], y["train"])
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))

In [37]:
delta_pred = IP(X, y, l2, "SST", thresh)
delta_pred.shape

(872, 2768)

In [54]:
# Loop over all dev points:
appro_ks = []
new_predictions = []
flip_list = []
for test_idx in tqdm(range(X["dev"].shape[0])):
    appro_k = approximate_k(test_idx, pred, delta_pred, y, thresh)
    if appro_k != None:
        change, _, prediction,new_prediction, error, top_k_index = new_train(appro_k, test_idx, delta_pred, l2, X, model, pred, y, thresh)
        print(test_idx, appro_k, pred[test_idx], new_prediction)
        appro_ks.append(appro_k)
        new_predictions.append(new_prediction)
        flip_list.append(top_k_index)
    else:
        appro_ks.append(None)
        new_predictions.append(None)
        flip_list.append(None)

appro_ks= np.array(appro_ks)
new_predictions=np.array(new_predictions)
flip_list = np.array(flip_list)
np.save("./results/" + "appro_ks_IP" + "_alg1_" + dataname  + str(l2) + ".npy", appro_ks)
np.save("./results/" + "new_predictions" + "_alg1_" + dataname + str(l2) + ".npy", new_predictions)
np.save("./results/" + "old_predictions" + "_alg1_" + dataname + str(l2) + ".npy", pred)
np.save("./results/" + "flip_list" + "_alg1_" + dataname + str(l2) + ".npy", flip_list)

  3%|███▊                                                                                                         | 30/872 [00:00<00:05, 149.17it/s]

0 134 [0.52077396] 0.49728713269858066
1 55 [0.48867849] 0.5025829044023986
2 151 [0.55569746] 0.504179411130753
3 97 [0.52066404] 0.49810821989098775
4 149 [0.54020816] 0.4999450458918281
5 186 [0.52770517] 0.4937434027218943
6 55 [0.51122005] 0.49961439710328226
7 3 [0.50068478] 0.49988251172861625
8 178 [0.53307483] 0.4978678339083847
9 100 [0.51799194] 0.49905691537294394
10 61 [0.51233241] 0.49912682859497276
11 11 [0.50222835] 0.4997309157782503
12 199 [0.54440747] 0.4992949226403268
13 20 [0.4938701] 0.5005344833735793
14 140 [0.52529472] 0.4983150198912873
15 53 [0.50777162] 0.49837402965294064
16 128 [0.52250313] 0.49723790928461015
17 19 [0.5064933] 0.49946195880727146
18 205 [0.54220615] 0.5001668728541306
19 81 [0.51278771] 0.497648166585883
20 22 [0.50394654] 0.4989553878449617
21 186 [0.55922473] 0.5009571418038667
22 117 [0.52156702] 0.49912520663704135
23 124 [0.53618588] 0.5000414442686401
24 18 [0.50573817] 0.4995815657887367
25 57 [0.51673559] 0.5000572911637763
26 1

  5%|█████▉                                                                                                       | 47/872 [00:00<00:05, 154.55it/s]

30 192 [0.53598534] 0.4988216535728453
31 39 [0.50757266] 0.49861882529365403
32 114 [0.52371276] 0.4993334452805755
33 20 [0.49619858] 0.5012053441259131
34 114 [0.51967975] 0.4984141754077538
35 14 [0.50313168] 0.49973362853377523
36 8 [0.49790666] 0.5003264305558184
37 86 [0.47104257] 0.5042566343217711
38 40 [0.50814262] 0.4984039870142087
39 13 [0.49678635] 0.5009759129592611
40 46 [0.52583782] 0.5000971645381274
41 49 [0.50868465] 0.498312332936441
42 107 [0.51980081] 0.4978765981332745
43 47 [0.51574345] 0.49999322746658276
44 30 [0.50522769] 0.4990541057302473
45 230 [0.55640552] 0.5027563859059556
46 8 [0.50171165] 0.4996183756359377
47 109 [0.52424709] 0.5000809132684233
48 127 [0.5249485] 0.49840715768261223
49 40 [0.50855993] 0.4993475711315648
50 77 [0.52841175] 0.4983113733426321
51 152 [0.53598209] 0.5001608488304656
52 155 [0.53189331] 0.4994926174690251
53 71 [0.51469578] 0.4989724111560665
54 134 [0.52986481] 0.49825247957515373
55 63 [0.51111193] 0.49804983595492525


  9%|█████████▉                                                                                                   | 79/872 [00:00<00:05, 152.64it/s]

...

 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 860/872 [00:05<00:00, 162.10it/s]

828 210 [0.54441463] 0.49723590687249153
829 84 [0.51672596] 0.4984766246746404
830 29 [0.51946619] 0.5004252450319798
831 44 [0.48745573] 0.5013577033461906
832 2 [0.50029319] 0.4997337486407822
833 208 [0.53678411] 0.49608175743709715
834 67 [0.51516344] 0.4994227024098077
835 24 [0.49340464] 0.5009196739304517
836 197 [0.44964339] 0.5058720769222067
837 16 [0.50303855] 0.4996356315385461
838 8 [0.49844626] 0.5003017698262878
839 29 [0.49100805] 0.5005887705889881
840 14 [0.49269979] 0.5006048105909631
841 28 [0.49224209] 0.5009162402168005
842 40 [0.49119474] 0.5010105528812432
843 75 [0.46490492] 0.4985638703026554
844 13 [0.49650564] 0.5003531796516806
845 126 [0.52291681] 0.49860086272908366
846 61 [0.51440413] 0.4994644450826649
847 14 [0.50297998] 0.499576671637271
848 45 [0.51241278] 0.5001918362787638
849 116 [0.47251305] 0.5026801708551717
850 97 [0.51852405] 0.4969270076343028
851 73 [0.47651716] 0.5009058398934219
852 2 [0.49919893] 0.5002201330771036
853 23 [0.50485461] 0

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 872/872 [00:05<00:00, 154.91it/s]

862 31 [0.50727646] 0.4993479737515096
863 154 [0.45015635] 0.5089639351643336
864 28 [0.50770905] 0.4993195831403888
865 77 [0.48459131] 0.5033954519981139
866 26 [0.50620115] 0.4990647806082188
867 123 [0.52284113] 0.49614285378999384
868 78 [0.4830456] 0.5033466379776133
869 35 [0.49325439] 0.5012591641927812
870 137 [0.52907722] 0.49864290021926855
871 10 [0.50165835] 0.499411554389577


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (872,) + inhomogeneous part.

In [65]:
new_predictions = new_predictions.reshape((872, 1))
np.sum(((new_predictions > 0.5) != (pred >0.5))) / 872

np.float64(0.9105504587155964)